In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklift.metrics import uplift_at_k
from sklift.models import SoloModel, TwoModels, ClassTransformation
from catboost import CatBoostClassifier

In [2]:
#Загрузим датафрейм
df_main = pd.read_csv('data/data.csv')
#Переименуем колонки
df_main.rename(columns = {'conversion' : 'target', 'offer' : 'treatment'}, inplace = True)
#Трансформируем Treatment
df_main['treatment'] = np.where(df_main['treatment'] != 'No Offer', 1, 0)
#Добавим колонку с id шниками как индексы
df_main['id'] = df_main.index
df_main.set_index('id',inplace=True)
df_main

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
id,,,,,,,,,
0,10,142.44,1,0,Surburban,0,Phone,1,0
1,6,329.08,1,1,Rural,1,Web,0,0
2,7,180.65,0,1,Surburban,1,Web,1,0
3,9,675.83,1,0,Rural,1,Web,1,0
4,2,45.34,1,0,Urban,0,Web,1,0
...,...,...,...,...,...,...,...,...,...
63995,10,105.54,1,0,Urban,0,Web,1,0
63996,5,38.91,0,1,Urban,1,Phone,1,0
63997,6,29.99,1,0,Urban,1,Phone,1,0


Мне кажется что сделать сначала feature feature engineering лучше, а потом уже разбивать, дробить данные и т.д, то есть данные должны быть сначала подготовленны, а потом уже работать с ними.<br>
Была мысль сделать фичи из "Offer", но оффер - целевое действие, которое нельзя проверять уже свершив его, поэтому признаки делать из него некорректно получается<br>
Смотря на датасет особо не понятно какие признаки можно делать, можно добавить признак категорию: Loyality_programm: 0, если вообще не использует никакие программы, 1 если использует или скидку или 1+1, но не обе сразу. И 2 если использует все возможные программы лояльности.<br>
Других вариантов для составления признаков особо не вижу

Колонку сделаю по такому плану: Создам ее со значением по умолчанию = 1, а потом по условиям, если оба поля лояльности = 0, то ее в ноль ставим, если оба в 1, то в 1.

In [3]:
def loyalty_encoder(df):
    df['loyalty_category'] = 1
    df.loc[(df.used_discount == 1) & (df.used_bogo == 1), 'loyalty_category'] = 2
    df.loc[(df.used_discount == 0) & (df.used_bogo == 0), 'loyalty_category'] = 0
    return df
#Добавляем новую категорию по лояльности
df_main = loyalty_encoder(df_main)
df_main

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target,loyalty_category
id,,,,,,,,,,
0,10,142.44,1,0,Surburban,0,Phone,1,0,1
1,6,329.08,1,1,Rural,1,Web,0,0,2
2,7,180.65,0,1,Surburban,1,Web,1,0,1
3,9,675.83,1,0,Rural,1,Web,1,0,1
4,2,45.34,1,0,Urban,0,Web,1,0,1
...,...,...,...,...,...,...,...,...,...,...
63995,10,105.54,1,0,Urban,0,Web,1,0,1
63996,5,38.91,0,1,Urban,1,Phone,1,0,1
63997,6,29.99,1,0,Urban,1,Phone,1,0,1


In [4]:
#Бьем на треин тест
train, test = train_test_split(df_main, test_size=0.3, random_state=42)
#Делаем данные для обучения
y_train = train.loc[train.index, 'target']
treat_train = train.loc[train.index, 'treatment']
train_features = train.drop(['target', 'treatment'], axis=1)
#То же самое нам нужно для теста
y_test = test.loc[test.index, 'target']
treat_test = test.loc[test.index, 'treatment']
test_features = test.drop(['target', 'treatment'], axis=1)

In [5]:
#Обозначим категориальные признаки
cat_features = ['zip_code', 'channel']
#Подготовим словарь под результаты
models_results = {
    'model': [],
    'uplift@10%': [],
    'uplift@20%': []
}
#В следующих ячейках уже обучаем модели

In [6]:
#Соло модель
sm = SoloModel(
    CatBoostClassifier(iterations=30, random_state=42, silent=True, cat_features=cat_features)
)

sm = sm.fit(train_features, y_train, treat_train)

uplift_sm = sm.predict(test_features)

sm10_score = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treat_test, strategy='by_group', k=0.1)
sm20_score = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treat_test, strategy='by_group', k=0.2)
models_results['model'].append('SoloModel')
models_results['uplift@10%'].append(sm10_score)
models_results['uplift@20%'].append(sm20_score)

In [7]:
#Трансформация классов
ct = ClassTransformation(
    CatBoostClassifier(iterations=30, random_state=42, silent=True, cat_features=cat_features)
)
ct = ct.fit(train_features, y_train, treat_train)

uplift_ct = ct.predict(test_features)

ct10_score = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treat_test, strategy='by_group', k=0.1)
ct20_score = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treat_test, strategy='by_group', k=0.2)

models_results['model'].append('ClassTransformation')
models_results['uplift@10%'].append(ct10_score)
models_results['uplift@20%'].append(ct20_score)

In [8]:
#Две независимые модели
tm = TwoModels(
    CatBoostClassifier(iterations=30, random_state=42, silent=True, cat_features=cat_features),
    CatBoostClassifier(iterations=30, random_state=42, silent=True, cat_features=cat_features),
    method='vanilla'  # независимые модели
)
tm = tm.fit(train_features, y_train, treat_train)

uplift_tm = tm.predict(test_features)

tm10_score = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treat_test, strategy='by_group', k=0.1)
tm20_score = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treat_test, strategy='by_group', k=0.1)
models_results['model'].append('TwoModels')
models_results['uplift@10%'].append(tm10_score)
models_results['uplift@20%'].append(tm20_score)


In [9]:
pd.DataFrame(data=models_results).sort_values('uplift@10%', ascending=False)



,model,uplift@10%,uplift@20%
0,SoloModel,0.110469,0.101803
1,ClassTransformation,0.105847,0.097598
2,TwoModels,0.091589,0.091589


Странно что у соломодели больше аплифт, может что-то сделал не так?